In [1]:
# The path to the local git repo for Indic NLP library
INDIC_NLP_LIB_HOME=r"/Users/mayuresh/Desktop/indic_nlp_library"

# The path to the local git repo for Indic NLP Resources
INDIC_NLP_RESOURCES=r"/Users/mayuresh/Desktop/indic_nlp_resources"

import pandas as pd
import numpy as np
import warnings
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append(r'{}'.format(INDIC_NLP_LIB_HOME))
from indicnlp import common
common.set_resources_path(INDIC_NLP_RESOURCES)
from indicnlp import loader
loader.load()


In [2]:
training_dataset = pd.read_csv("/Users/mayuresh/Desktop/MOLD/Mold/MOLD_Training2.csv")
training_dataset.head()
training_dataset.dropna()
training_dataset['subtask_c'].fillna("NULL")

0       GRP
1       GRP
2       GRP
3       GRP
4       GRP
       ... 
2493    OTH
2494    OTH
2495    OTH
2496    OTH
2497    OTH
Name: subtask_c, Length: 2498, dtype: object